In [2]:
from ibl_pipeline.analyses import behavior
from ibl_pipeline import behavior as behavior_ingest
from ibl_pipeline import subject, acquisition
from ibl_pipeline.utils import psychofit as psy
from uuid import UUID
import numpy as np
import datetime
import datajoint as dj
import json
import seaborn as sns
import pandas as pd
import plotly
from plotly import tools
import plotly.graph_objs as go
import statsmodels.stats.proportion as smp
plotly.offline.init_notebook_mode(connected=True)

In [75]:
subj = subject.Subject & 'subject_nickname="IBL_46"'
# get fit parameters by date
fit_pars = (behavior.BehavioralSummaryByDate.PsychResults * subj).proj(
    'session_date', 'prob_left','threshold', 'bias', 'lapse_low', 'lapse_high').fetch(as_dict=True)
fit_pars = pd.DataFrame(fit_pars)

In [76]:
par_names = ['threshold', 'bias', 'lapse_low', 'lapse_high']

pars = dict()
for par_name in par_names:
    pars[par_name] = []

prob_lefts = fit_pars['prob_left'].unique()
    
for prob_left in prob_lefts:
    prob_left_filter = fit_pars['prob_left']==prob_left
    if prob_left == 0.2:
        dot_color = 'orange'
    elif prob_left == 0.5:
        dot_color = 'black'
    elif prob_left == 0.8:
        dot_color = 'cornflowerblue'
    else:
        dot_color = 'gray'
    
    fit_pars_sub = fit_pars[prob_left_filter]
    
    for ipar, par_name in enumerate(par_names):
        if ipar==0:
            show_legend = True
        else:
            show_legend = False
        pars[par_name].append(
            go.Scatter(      
                x=[t.strftime('%Y-%m-%d') for t in fit_pars_sub['session_date'].tolist()],
                y=fit_pars_sub[par_name].tolist(),
                mode='markers',
                marker=dict(
                    size=5,
                    color=dot_color),
                name=f'p_left = {prob_left}',
                xaxis='x{}'.format(4-ipar),
                yaxis='y{}'.format(4-ipar),
                showlegend=show_legend
            )
        )


In [77]:
pars_data = [pars[par_name][i] for i, prob_left in enumerate(prob_lefts)
             for par_name in par_names]

layout = go.Layout(
    xaxis1=dict(
        domain=[0, 1],
        title='Date'
    ),
    yaxis1=dict(
        domain=[0, 0.2],
        anchor='x1',
        range=[-0.02, 1.02],
        title='$Lapse high\ (\\lambda)$'
    ),
    xaxis2=dict(
        domain=[0, 1],
    ),
    yaxis2=dict(
        domain=[0.25, 0.45],
        anchor='x2',
        range=[-0.02, 1.02],
        title='$Lapse low\ (\\gamma)$'
    ),
    xaxis3=dict(
        domain=[0, 1],  
    ),
    yaxis3=dict(
        domain=[0.5, 0.7],
        anchor='x3',
        range=[-105, 105],
        title='$Bias\ (\\mu)$'
    ),
    xaxis4=dict(
        domain=[0, 1],
    ),
    yaxis4=dict(
        domain=[0.75, 1],
        anchor='x4',
        range=[-5, 105],
        title='$Threshold\ (\\sigma)$'
    ),
    height=1000, 
    width=500,
    title='Fit Parameters',
)

fig = go.Figure(data=pars_data, layout=layout)

plotly.offline.iplot(fig)


In [71]:
f = open('fit_pars.json', 'w')
s = json.dumps(fig.to_plotly_json())
f.write(s)
f.close()